In [8]:
using Convex
tic()

# set data
k = 5; # number of elements
n = 20; # number of antennas at BS
P0 = 100; # max power budget
P_cir = 100;
r0 = 0.2; # data rate QoS
delta = 1; # normalized Gaussian noise

# set channel model from a BS (multi-antenna) to multiple UEs (single-antenna)
H = 0.5*(randn(n,k) + im*randn(n,k))

# Assume pathloss (large-scale) channel is fixed with the value of 1e-2
path = 1e-2;

# Applying Conjugate beamforming
F_conj = H*inv(H'*H);

# Nomalized vector beamforming
f_nor = randn(n,k) + im*randn(n,k);
for i=1:k
    f_nor[:,i] = F_conj[:,i]/norm(F_conj[:,i],2);
end

beta = Array{Any,2}(rand(k,k))
for i = 1:k
    beta[:,i] = [path*abs2(H[:,i]'*H[:,j])/norm(H[:,j]) for j = 1:k]
end


# ##############################################
# ########## Initial point ############
p_ini = Variable(k , Positive())
obj_ini = sumsquares(p_ini) + P_cir

problem = minimize(obj_ini)

# problem.constraints += sumsquares(p_ini) <= P0
for i = 1:k
    term_j = Variable(k , Positive());
    for j = 1:k
        if (j!=i)
            term_j = [p_ini[j]*sqrt(beta[j,i]) for (j!=i) = 1:k] # [term_j, p_ini[j]*sqrt(beta[j,i])]
        end
    end
    @show size(term_j)
    problem.constraints += p_ini[i]*sqrt(beta[i,i]/delta) >= sqrt(exp(1)^r0-1)*norm(term_j,2)
    # problem.constraints += p_ini[i]*sqrt(beta[i,i]/delta) >= sqrt(exp(1)^r0-1) 
end

using ECOS
@time solve!(problem, ECOSSolver())

pow_ini = p_ini.value
@show pow_ini

size(term_j) = (5,)


LoadError: MethodError: no method matching isnan(::Convex.EucNormAtom)[0m
Closest candidates are:
  isnan([1m[31m::BigFloat[0m) at mpfr.jl:796
  isnan([1m[31m::Float16[0m) at float16.jl:117
  isnan([1m[31m::AbstractFloat[0m) at float.jl:355
  ...[0m